In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer

from dataset.tokenize import TokenizeMapWrapper, Seq2SeqTokenizeMapWrapper

2023-05-14 17:54:43.090950: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 17:54:45.917512: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/wsl/lib:
2023-05-14 17:54:45.918735: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/wsl/lib:
2023-05-14 17:54:45.918753: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [2]:
df = pd.read_json('./data/dataset-term-summary.json', encoding='utf-8')
print(df['summary'][0])
df.head()

제7조 (근저당권 설정)을은 제4조 대금지급의무 기타 본 계약상의 의무 이행을 담보하기 위하여, 갑과의 합의에따른 부동산 위에 갑을 근저당권자로 하는 채권 최고금액   원의 근저당권을 설정하여야 한다.
2. 하자보증 기간이 경과한 후 계약물품에 고장이 발생하는 경우 갑은 을의 요청에 따라 물품을 수리하는데 필요한부품을 을에게 공급한다.
5. 본조에 해당하는 사유로 계약이 해제될 경우에 을이 기납부한 공급대금에서 총 공급대금의10퍼센트는 위약금으로 갑에게 귀속된다.


,index,text,title,id,label,summary
0,0,제 1 조 (계약의 목적)\n본 계약은 을이 갑의 000 상품을 상호 협의한 할인매...,공급계약,199,disadvantageous,제7조 (근저당권 설정)을은 제4조 대금지급의무 기타 본 계약상의 의무 이행을 담보...
1,1,제1조 (목 적)\n000(이하 요양센터)는 시설에 입소한 이용자에 대하여 노인장기...,"입소,입주,입점계약",84,disadvantageous,제1조 (목 적)000(이하 요양센터)는 시설에 입소한 이용자에 대하여 노인장기요양...
2,2,갑종부동산관리신탁계약서 \n \n \n위탁자 (이하 위탁자 라 한다)는(은) ...,예식업,166,disadvantageous,제1조(목적) 이 약관은 예식장을 운영하는 사업자(이하 사업자 라 합니다)와 예식...
3,3,"제1조(계약금액 및 납부기한)\n1. 을 은 계약과 동시에 계약금을 지불하고, ...",분양계약,36,disadvantageous,(1) 계약금(2) 중도금 1차(3) 중도금 2차(4) 잔금제2조(계약의 해제)1....
4,4,제1조 (분양금액)\n위 표시 물건의 분양금액은 금 ( )원정(부가...,분양계약,28,disadvantageous,2. 제2조 제2항에 해당하는 사유로 본 계약이 해제된 때에는 갑 은 을 에게 ...


### Preprocessing

### Tokenizing

In [ ]:
checkpoint = ''

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenizer_map_wrapper = Seq2SeqTokenizeMapWrapper(tokenizer, 'text', 'summary')

In [3]:
droped_df = df.drop(columns=['id', 'title', 'index', 'label'], axis=1)
droped_df

,text,summary
0,제 1 조 (계약의 목적)\n본 계약은 을이 갑의 000 상품을 상호 협의한 할인매...,제7조 (근저당권 설정)을은 제4조 대금지급의무 기타 본 계약상의 의무 이행을 담보...
1,제1조 (목 적)\n000(이하 요양센터)는 시설에 입소한 이용자에 대하여 노인장기...,제1조 (목 적)000(이하 요양센터)는 시설에 입소한 이용자에 대하여 노인장기요양...
2,갑종부동산관리신탁계약서 \n \n \n위탁자 (이하 위탁자 라 한다)는(은) ...,제1조(목적) 이 약관은 예식장을 운영하는 사업자(이하 사업자 라 합니다)와 예식...
3,"제1조(계약금액 및 납부기한)\n1. 을 은 계약과 동시에 계약금을 지불하고, ...",(1) 계약금(2) 중도금 1차(3) 중도금 2차(4) 잔금제2조(계약의 해제)1....
4,제1조 (분양금액)\n위 표시 물건의 분양금액은 금 ( )원정(부가...,2. 제2조 제2항에 해당하는 사유로 본 계약이 해제된 때에는 갑 은 을 에게 ...
...,...,...
7995,00000 약관 \n\n \n제1관 목적 및 용어의 정의 \n \n제1조 ( 목적 ...,중요한 사항 계약 전 알릴 의무와 관련하여 회사가 그 사실을 알았더라면 계약의 청...
7996,여신거래 약관\n이 여신금융회사 여신거래기본약관(이하 약관 이라 합니다)은 000...,채무자가 금융회사에 대한 채무의 이행을 지체한 경우에는 곧 지급기로 한 금액에 대하...
7997,1. 총칙 \n이 서비스 약관은 회원이 000(이하 병원 )에서 제공하는 서비스(...,1. 총칙이 서비스 약관은 회원이 000(이하 병원 )에서 제공하는 서비스(이하서...
7998,주택화재보험 보통약관 \n\n 제1조(보험계약의 성립) \n ①보험계약은 보험계약자...,제2항에 따라 계약이 취소된 경우에는 회사는 이미 납입한 보험료를 계약자에게 돌려 ...


In [4]:
dataset = Dataset.from_pandas(droped_df)
dataset

Dataset({
    features: ['text', 'summary', '__index_level_0__'],
    num_rows: 7943
})

In [ ]:
dataset = dataset.map(tokenizer_map_wrapper)
pprint(dataset.select(range(1)))